# Flights passengers example
## Imports

In [ ]:
import torch
import numpy as np
import calendar
from datetime import timedelta, datetime
from matplotlib import pyplot as plt

from time_series_predictor import TimeSeriesPredictor
from time_series_models import BenchmarkLSTM
from flights_time_series_dataset import FlightsDataset, convert_year_month_array_to_datetime

## Config

In [ ]:
plot_config = {}
plot_config['training progress'] = True
plot_config['prediction on training data'] = True
plot_config['prediction on validation data'] = True
plot_config['forecast'] = True

forecast_config = {}
forecast_config['include history'] = True
forecast_config['months ahead'] = 24

predictor_config = {}
predictor_config['epochs'] = 1000
predictor_config['learning rate'] = 1e-2
predictor_config['hidden dim'] = 100
predictor_config['layers num'] = 3

config = {}
config['plot'] = plot_config
config['forecast'] = forecast_config
config['predictor'] = predictor_config
config['predict on training data enabled'] = True
config['forecast enabled'] = True

## Time Series Predictor instantiation

In [ ]:
tsp = TimeSeriesPredictor(
    BenchmarkLSTM(
        hidden_dim=config['predictor']['hidden dim'],
        num_layers=config['predictor']['layers num']
    ),
    lr=config['predictor']['learning rate'],
    max_epochs=predictor_config['epochs'],
    train_split=None,
    optimizer=torch.optim.Adam
)

## Training process

In [ ]:
ds = FlightsDataset()
tsp.fit(ds)

### Plot training evolution

In [ ]:
if config['plot']['training progress']:
    history_length = len(tsp.ttr.regressor_['regressor'].history)
    train_loss = np.zeros((history_length, 1))
    for epoch in tsp.ttr.regressor_['regressor'].history:
        epoch_number = epoch['epoch']-1
        train_loss[epoch_number] = epoch['train_loss']
    _, axes_one = plt.subplots(figsize=(20, 20))
    axes_one.plot(train_loss, 'o-', label='training')
    axes_one.set_xlabel('Epoch')
    axes_one.set_ylabel('MSE')
    plt.legend()

## Prediction on training data

In [ ]:
if config['predict on training data enabled']:
    dataloader = tsp.ttr.regressor['regressor'].get_iterator(tsp.dataset)
    x, y = dataloader.dataset[:]
    netout = tsp.predict(x)
    d_output = netout.shape[-1]
    if config['plot']['prediction on training data']:
        fig, axs = plt.subplots(d_output, 1, figsize=(20,20))
        axs = [axs]
    idx_range = [np.random.randint(0, len(tsp.dataset))]
    for idx in idx_range:
        if config['plot']['prediction on training data']:
            x_absciss = convert_year_month_array_to_datetime(x[idx, :, :])
        for idx_output_var in range(d_output):
            # Select real passengers data
            y_true = y[idx, :, idx_output_var]

            y_pred = netout[idx, :, idx_output_var]
            if config['plot']['prediction on training data']:
                ax = axs[idx_output_var]
                ax.plot(x_absciss, y_true, label="Truth", color='tab:blue')
                ax.plot(x_absciss, y_pred, label="Prediction", color='tab:orange')
                if idx == idx_range[0]:
                    ax.set_title(tsp.dataset.labels['y'][idx_output_var] + ' over time')
                    ax.set_xlabel('date')
                    ax.set_ylabel(tsp.dataset.labels['y'][idx_output_var])
                    ax.legend()
    if config['plot']['prediction on training data']:
        plt.show()

## Future forecast

In [ ]:
# def make_x_pred(x, n_months_ahead, include_history=True):
#     def raw_add_months(sourcedate, n_months):
#         month = sourcedate.month - 1 + n_months
#         year = sourcedate.year + month // 12
#         month = month % 12 + 1
#         day = min(sourcedate.day, calendar.monthrange(year,month)[1])
#         return datetime(year, month, day)
#     def add_months(months, n_months):
#         return [raw_add_months(month, n_months) for month in months]
#     def add_months_2(month, n_months):
#         return [raw_add_months(month, n_month) for n_month in n_months]
#     x_dates = convert_year_month_array_to_datetime(x.squeeze())
#     last_month = x_dates[-1:][0]
#     next_n_months = add_months_2(last_month, np.arange(n_months_ahead)+1)
#     if include_history:
#         n_months = x_dates + next_n_months
#     else:
#         n_months = next_n_months
#     x_pred = np.array([[dt.month, dt.year] for dt in n_months]).reshape(1, -1, 2).astype(np.float32)
#     return x_pred

if config['forecast enabled']:
    dataloader = tsp.ttr.regressor['regressor'].get_iterator(tsp.dataset)
    x, y = dataloader.dataset[:]
    # x_pred = make_x_pred(x, config['forecast']['months ahead'])
    # netout = tsp.predict(x_pred)
    netout, x_pred = tsp.forecast(config['forecast']['months ahead'], include_history = config['forecast']['include history'])
    d_output = netout.shape[-1]
    if config['plot']['forecast']:
        fig, axs = plt.subplots(d_output, 1, figsize=(20,20))
        axs = [axs]
    idx_range = [np.random.randint(0, len(tsp.dataset))]
    for idx in idx_range:
        if config['plot']['forecast']:
            x_absciss = convert_year_month_array_to_datetime(x[idx, :, :])
            x_pred_absciss = convert_year_month_array_to_datetime(x_pred[idx, :, :])
        for idx_output_var in range(d_output):
            # Select real passengers data
            y_true = y[idx, :, idx_output_var]

            y_pred = netout[idx, :, idx_output_var]
            if config['plot']['prediction on validation data']:
                ax = axs[idx_output_var]
                ax.plot(x_pred_absciss, y_pred, label="Prediction", color='tab:orange')
                ax.plot(x_absciss, y_true, label="Truth", color='tab:blue')
                if idx == idx_range[0]:
                    ax.set_title(tsp.dataset.labels['y'][idx_output_var] + ' over time')
                    ax.set_xlabel('date')
                    ax.set_ylabel(tsp.dataset.labels['y'][idx_output_var])
                    ax.legend()
    if config['plot']['prediction on validation data']:
        plt.show()